In [3]:
%%capture
!pip install dmi-open-data

In [83]:
from datetime import datetime
import os
from dmi_open_data import DMIOpenDataClient, Parameter, ClimateDataParameter
from tqdm.notebook import tqdm
import pandas as pd

See: https://confluence.govcloud.dk/pages/viewpage.action?pageId=26476690#UserCreation&AccessManagement-RegisterAsAUser

To obtain an API key for DMI 

In [16]:
API_KEY = 'dea6495e-4b55-44c1-9be1-example'
climate_data_client = DMIOpenDataClient(api_key=API_KEY)

In [14]:
parameters = climate_data_client.list_parameters()

In [15]:
parameters

[{'name': 'TempDry',
  'value': 'temp_dry',
  'enum': <Parameter.TempDry: 'temp_dry'>},
 {'name': 'TempDew',
  'value': 'temp_dew',
  'enum': <Parameter.TempDew: 'temp_dew'>},
 {'name': 'TempMeanPast1h',
  'value': 'temp_mean_past1h',
  'enum': <Parameter.TempMeanPast1h: 'temp_mean_past1h'>},
 {'name': 'TempMaxPast1h',
  'value': 'temp_max_past1h',
  'enum': <Parameter.TempMaxPast1h: 'temp_max_past1h'>},
 {'name': 'TempMinPast1h',
  'value': 'temp_min_past1h',
  'enum': <Parameter.TempMinPast1h: 'temp_min_past1h'>},
 {'name': 'TempMaxPast12h',
  'value': 'temp_max_past12h',
  'enum': <Parameter.TempMaxPast12h: 'temp_max_past12h'>},
 {'name': 'TempMinPast12h',
  'value': 'temp_min_past12h',
  'enum': <Parameter.TempMinPast12h: 'temp_min_past12h'>},
 {'name': 'TempGrass',
  'value': 'temp_grass',
  'enum': <Parameter.TempGrass: 'temp_grass'>},
 {'name': 'TempGrassMaxPast1h',
  'value': 'temp_grass_max_past1h',
  'enum': <Parameter.TempGrassMaxPast1h: 'temp_grass_max_past1h'>},
 {'name': 

In [18]:
closest_station = climate_data_client.get_closest_station(
                    latitude=55.707722,
                    longitude=12.562119)

In [19]:
closest_station

{'geometry': {'coordinates': [12.562, 55.716], 'type': 'Point'},
 'id': '3a54f57a-f1b6-463d-98a5-bf8a590992bf',
 'type': 'Feature',
 'properties': {'barometerHeight': 9.8,
  'country': 'DNK',
  'created': '2021-02-11T16:25:51Z',
  'name': 'Dmi',
  'operationFrom': '1974-01-01T00:00:00Z',
  'operationTo': None,
  'owner': 'DMI',
  'parameterId': ['cloud_cover',
   'cloud_height',
   'humidity',
   'humidity_past1h',
   'leav_hum_dur_past10min',
   'leav_hum_dur_past1h',
   'precip_dur_past10min',
   'precip_dur_past1h',
   'precip_past10min',
   'precip_past1h',
   'precip_past1min',
   'precip_past24h',
   'pressure',
   'pressure_at_sea',
   'temp_dew',
   'temp_dry',
   'temp_max_past12h',
   'temp_max_past1h',
   'temp_mean_past1h',
   'temp_min_past12h',
   'temp_min_past1h',
   'visib_mean_last10min',
   'visibility',
   'weather',
   'wind_dir',
   'wind_dir_past1h',
   'wind_gust_always_past1h',
   'wind_max',
   'wind_max_per10min_past1h',
   'wind_min',
   'wind_min_past1h',
 

In [20]:
closest_station['properties']['stationId']

'06184'

In [37]:
observations = climate_data_client.get_observations(
    parameter=Parameter.TempDry,
    station_id=closest_station['properties']['stationId'],
    from_time=datetime(2020, 2, 26),
    to_time=datetime(2021, 11, 2),
    limit=100000)

Observations are in a https://geojson.org/ format

In [39]:
observations

[{'geometry': {'coordinates': [12.562, 55.716], 'type': 'Point'},
  'id': 'a4d2353d-9b62-9fef-954d-6c4baf008622',
  'type': 'Feature',
  'properties': {'created': '2021-11-01T23:56:38.258443Z',
   'observed': '2021-11-02T00:00:00Z',
   'parameterId': 'temp_dry',
   'stationId': '06184',
   'value': 10.3}},
 {'geometry': {'coordinates': [12.562, 55.716], 'type': 'Point'},
  'id': 'b01a15bf-ecde-6355-1e34-924721d96cc2',
  'type': 'Feature',
  'properties': {'created': '2021-11-01T23:46:12.172205Z',
   'observed': '2021-11-01T23:50:00Z',
   'parameterId': 'temp_dry',
   'stationId': '06184',
   'value': 10.3}},
 {'geometry': {'coordinates': [12.562, 55.716], 'type': 'Point'},
  'id': '7bc0b922-13f5-bc5a-7c5b-f5706298450c',
  'type': 'Feature',
  'properties': {'created': '2021-11-01T23:36:17.666951Z',
   'observed': '2021-11-01T23:40:00Z',
   'parameterId': 'temp_dry',
   'stationId': '06184',
   'value': 10.3}},
 {'geometry': {'coordinates': [12.562, 55.716], 'type': 'Point'},
  'id': '9

In [72]:
temp = []
dates = []

for obs in tqdm(observations):
    date = obs['properties']['observed']
    val = obs['properties']['value']
    temp.append(val)
    dates.append(date)
    

  0%|          | 0/88469 [00:00<?, ?it/s]

In [77]:
len(dates) == len(temp)

True

In [80]:
data = {'Date' : dates,
       'Temperature' : temp}

In [84]:
pd.DataFrame(data)

,Date,Temperature
0,2021-11-02T00:00:00Z,10.3
1,2021-11-01T23:50:00Z,10.3
2,2021-11-01T23:40:00Z,10.3
3,2021-11-01T23:30:00Z,10.4
4,2021-11-01T23:20:00Z,10.2
...,...,...
88464,2020-02-26T00:40:00Z,1.9
88465,2020-02-26T00:30:00Z,2.0
88466,2020-02-26T00:20:00Z,2.0
88467,2020-02-26T00:10:00Z,2.0
